In [1]:
import pandas as pd


In [2]:

player_df = pd.read_csv('./data/player_data_04-2020.csv')
player_df.head()

,Name,Equip1,Equip2,Equip3,Equip4,Equip5,Equip6,Equip7,Equip8,Equip9,...,Mining,Herblore,Agility,Thieving,Slayer,Farming,Runecrafting,Hunter,Construction,Location
0,Aweb0kz,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,60,1,1,1,1,1,1,1,1,ge
1,Popeye2018,-1,-1,1660,853,-1,-1,-1,1099,-1,...,83,1,10,1,1,1,30,1,1,ge
2,ImFatNotCute,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,41,30,25,20,63,17,9,1,1,ge
3,PappoSeco,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,41,1,1,1,1,1,1,1,1,ge
4,GB Justin,-1,-1,-1,1381,-1,-1,-1,-1,-1,...,1,1,37,19,33,1,1,1,27,ge
